In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta

# Load appointment data
df = pd.read_csv(r"C:\Users\SAI RIZWAN\Downloads\synthetic_patient_flow.csv")  # Ensure correct dataset format

# Convert time columns to datetime
df['scheduled_time'] = pd.to_datetime(df['scheduled_time'])
df['actual_time'] = pd.to_datetime(df['actual_time'])

# Calculate delay in minutes
df['delay'] = (df['actual_time'] - df['scheduled_time']).dt.total_seconds() / 60  
df['hour'] = df['scheduled_time'].dt.hour
df['day_of_week'] = df['scheduled_time'].dt.dayofweek

# Additional Features
df['patients_waiting'] = df.groupby('doctor_id')['scheduled_time'].rank(method="first")  # Number of patients waiting
df['previous_doctor_delay'] = df.groupby('doctor_id')['delay'].shift(1).fillna(0)  # Previous patient's delay

# Define features and target variable
features = ['doctor_id', 'hour', 'day_of_week', 'patients_waiting', 'previous_doctor_delay']
target = 'delay'

# Train AI Model
X = df[features]
y = df[target]

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Evaluate Model Performance
y_pred = model.predict(X)
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.2f} minutes")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} minutes")

# Function to predict wait time & handle early arrivals
def predict_wait_time(doctor_id, scheduled_time, actual_arrival_time):
    hour = scheduled_time.hour
    day_of_week = scheduled_time.weekday()
    
    input_data = pd.DataFrame([[doctor_id, hour, day_of_week, 1, 5]],  
                              columns=['doctor_id', 'hour', 'day_of_week', 'patients_waiting', 'previous_doctor_delay'])
    predicted_delay = model.predict(input_data)[0]  # Predicted delay in minutes
    
    # Adjust for early arrival (if applicable)
    early_arrival_buffer = 15  # Allow flexibility for early patients
    if actual_arrival_time <= scheduled_time - timedelta(minutes=early_arrival_buffer):
        predicted_delay = max(5, predicted_delay - early_arrival_buffer)  # Minimum wait of 5 minutes

    return predicted_delay

# Function to dynamically reschedule appointment
def reschedule_appointment(doctor_id, scheduled_time, actual_arrival_time):
    doctors = df['doctor_id'].unique()
    best_doctor = doctor_id
    best_wait_time = predict_wait_time(doctor_id, scheduled_time, actual_arrival_time)
    
    for doc in doctors:
        if doc != doctor_id:
            new_wait_time = predict_wait_time(doc, scheduled_time, actual_arrival_time)
            if 5 <= new_wait_time < best_wait_time:  # Ensure minimum wait of 5 min
                best_doctor = doc
                best_wait_time = new_wait_time
    
    return best_doctor, best_wait_time

# Example usage: A patient arrives early
scheduled_appointment = datetime(2024, 3, 26, 18, 30)
actual_arrival = datetime(2024, 3, 26, 18, 10)  # 20 minutes early

best_doctor, predicted_wait = reschedule_appointment(doctor_id=5, scheduled_time=scheduled_appointment, actual_arrival_time=actual_arrival)

# Notify Patient
print(f"📩 Notification to Patient 101:")
print(f"🔔 Your estimated wait time is {predicted_wait:.2f} minutes.")
print(f"✅ Please be ready for your consultation.")
print(f"✅ Patient assigned to Doctor {best_doctor} with new expected wait time: {predicted_wait:.2f} minutes")




Model Performance:
Mean Absolute Error (MAE): 6.49 minutes
Root Mean Squared Error (RMSE): 7.78 minutes
📩 Notification to Patient 101:
🔔 Your estimated wait time is 5.00 minutes.
✅ Please be ready for your consultation.
✅ Patient assigned to Doctor 5 with new expected wait time: 5.00 minutes
